# Bài tập về Sentiment Analysis
Mục tiêu của bài toán Sentiment Analysis (SA) là phân loại được ý kiến bên trong một đoạn văn bản. Ý kiến này có thể chỉ là binary (tốt-xấu) hoặc multi-class (thang điểm từ 1-5).

## Data
Trong bài tập này, chúng ta sử dụng dữ liệu Review được tổng hợp từ Amazon trong một category nhất định. Download nó bằng lệnh `!wget` với đường link như sau:

In [ ]:
### Amazon, retrieved from seemingly authentic source: http://jmcauley.ucsd.edu/data/amazon/
#metadata only !wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Amazon_Instant_Video.csv
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Digital_Music_5.json.gz -O ./reviews_Digital_Music_5.json.gz

--2021-05-28 10:41:31--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Digital_Music_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32052614 (31M) [application/x-gzip]
Saving to: ‘./reviews_Digital_Music_5.json.gz’

./reviews_Digital_M 100%[===================>]  30.57M  3.23MB/s    in 13s     

2021-05-28 10:41:44 (2.34 MB/s) - ‘./reviews_Digital_Music_5.json.gz’ saved [32052614/32052614]



Dữ liệu được trả về dưới dạng json.gz, là json nhưng không phải là một file text thông thường. Import thư viện `gzip` (cài nếu không có bằng pip), và sử dụng code bên dưới để load dữ liệu vào một `DataFrame` của pandas.

In [ ]:
import pandas as pd
import json, gzip

def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

data = pd.DataFrame(parse('./reviews_Digital_Music_5.json.gz'))   
data.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,5555991584,"[3, 3]",5.0,"It's hard to believe ""Memory of Trees"" came ou...","09 12, 2006",A3EBHHCZO6V2A4,"Amaranth ""music fan""",Enya's last great album,1158019200
1,5555991584,"[0, 0]",5.0,"A clasically-styled and introverted album, Mem...","06 3, 2001",AZPWAXJG9OJXV,bethtexas,Enya at her most elegant,991526400
2,5555991584,"[2, 2]",5.0,I never thought Enya would reach the sublime h...,"07 14, 2003",A38IRL0X2T4DPF,bob turnley,The best so far,1058140800
3,5555991584,"[1, 1]",5.0,This is the third review of an irish album I w...,"05 3, 2000",A22IK3I6U76GX0,Calle,Ireland produces good music.,957312000
4,5555991584,"[1, 1]",4.0,"Enya, despite being a successful recording art...","01 17, 2008",A1AISPOIIHTHXX,"Cloud ""...""",4.5; music to dream to,1200528000


## Preprocessing & Vectorizing
Dữ liệu text đang có là một string chưa được phân tích bằng bất cứ thứ gì. Chúng ta không thể bỏ dữ liệu này trực tiếp vào các Vectorizer của sklearn; mà có 2 lựa chọn:
 - Biến đổi trường "reviewText" sang dạng mà Vectorizer có thể tách được bằng hàm `.split` (e.g "Who is that?" -> "Who is that ?")
 - Sử dụng một hàm tự xây dựng, có thể dưới dạng `lambda x: y` có khả năng tách được các text thành các từ token, và đặt hàm này vào argument `tokenizer` của các Vectorizer (e.g "Who is that" -> \["Who", "is", "that", "?"\])

**Học viên cần sử dụng một trong 2 phương pháp trên kết hợp với các phần đã học trên lớp (TF-IDF, n-gram), Vector hóa dữ liệu ra ma trận sparse vào biến `matrix_data`:**

In [ ]:
# Example sử dụng nltk
from nltk.tokenize import sent_tokenize, word_tokenize
sent_tokenize(data.loc[0]["reviewText"]) # Tách thành các câu đơn

['It\'s hard to believe "Memory of Trees" came out 11 years ago;it has held up well over the passage of time.It\'s Enya\'s last great album before the New Age/pop of "Amarantine" and "Day without rain."',
 'Back in 1995,Enya still had her creative spark,her own voice.I agree with the reviewer who said that this is her saddest album;it is melancholy,bittersweet,from the opening title song.',
 '"Memory of Trees" is elegaic&majestic.',
 ';"Pax Deorum" sounds like it is from a Requiem Mass,it is a dark threnody.Unlike the reviewer who said that this has a "disconcerting" blend of spirituality&sensuality;,I don\'t find it disconcerting at all.',
 '"Anywhere is" is a hopeful song,looking to possibilities.',
 '"Hope has a place" is about love,but it is up to the listener to decide if it is romantic,platonic,etc.I\'ve always had a soft spot for this song.',
 '"On my way home" is a triumphant ending about return.This is truly a masterpiece of New Age music,a must for any Enya fan!']

In [ ]:
# Example sử dụng spacy
import spacy
spacy_tokenizer = spacy.load('en')
spacy_tokenizer(data.loc[0]["reviewText"])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def full_tokenizer(raw):
    sentences = sent_tokenize(raw) # Tách thành các câu đơn
    tokens = (word_tokenize(l.strip()) for l in sentences) # Tách thành các Token
    return [w for l in tokens for w in l]
# Chuyển sang dạng vector
vectorizer = TfidfVectorizer(tokenizer=full_tokenizer, ngram_range=(1, 3), max_df=0.95, min_df=3)
matrix_data = vectorizer.fit_transform(data["reviewText"])

In [ ]:
matrix_data.shape

(64706, 1164448)

## Building Model
Tương tự như đã làm trên lớp, chúng ta cần phải tách dữ liệu ra tập train và test để có thể kiểm soát chất lượng mô hình

**Học viên tách các bộ train và test vào các cặp (features, labels): (`X_train`, `y_train`), (`X_test`, `y_test`) với tỉ lệ 8:2 và random_state 42**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(matrix_data, data["overall"].astype(int), test_size=0.2, random_state=42)

Chúng ta sử dụng mô hình Classification phù hợp, có thể là Naive Bayes, SVM hoặc LogisticRegression.

**Học viên tạo mô hình và train cho mô hình đó bằng dữ liệu train:**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, NuSVC, SVC
#model = LogisticRegression(C=1.0, multi_class='auto', max_iter=1000, random_state=0, solver='lbfgs', n_jobs=5)
#model = LinearSVC(random_state=0, class_weight="balanced", max_iter=1000)
model = SVC(gamma="auto", random_state=0, class_weight="balanced")
model.fit(X_train, y_train)

## Evaluation
Đầu tiên, chúng ta sử dụng các metrics và tính qua các giá trị đánh giá thường dùng.

**Tính kết quả của mô hình qua giá trị F1 và accuracy:**

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
y_pred = model.predict(X_test)
print("F1: {:.4f}; Acc: {:.4f}".format(f1_score(y_test, y_pred, average="weighted"), accuracy_score(y_test, y_pred)))

Với các mô hình classification, chúng ta có thể in ra các giá trị ROC, đại biểu một phần cho chất lượng của classifier bên trong - mô hình với đường cong càng cao, area càng lớn, thì khả năng chọn được một giá trị tốt để phân biệt class đó với các class khác càng cao.

In [ ]:
from sklearn.metrics import roc_curve, auc

n_classes = len(model.classes_)
#print(n_classes)
y_score = model.predict_proba(X_test) # Tính accuracy
fpr = dict()
tpr = dict()
roc_auc = dict()
# Tính true positive rate, false positive rate và AUC
for i in range(n_classes):
    class_idx = model.classes_[i]
    fpr[i], tpr[i], _ = roc_curve(y_test == class_idx, y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Print out đồ thị các giá trị vừa tính toán
import matplotlib.pyplot as plt
plt.figure()
lw = 2
for i, color in zip(range(n_classes), ["darkorange", "blue", "green", "teal", "red", "yellow"]):
    class_idx = model.classes_[i]
    plt.plot(fpr[i], tpr[i], color=color,
         lw=lw, label='ROC curve for class %d (area = %0.2f)' % (class_idx, roc_auc[i]))
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

Một thông tin khác có thể sử dụng là confusion matrix, có thể dùng để visualize kết quả trả ra và lượng lệch của chúng với kết quả chính xác. Code bên dưới in ra confusion matrix với các hàng là nhãn đúng (True label) và các cột là nhãn được đoán ra (Predicted label). VD: hàng 4 cột 1 đại biểu cho bao nhiêu review có nhãn đúng là 4 nhưng mô hình đoán nhãn 1.

**Học viên phát hiện gì khi in ra confusion matrix ở dưới? Vấn đề gọi là gì và xử lý như thế nào?**

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
confmx = confusion_matrix(y_pred, y_test, labels=reg_model.classes_)
classes = reg_model.classes_
cmap = plt.cm.Blues
#print(confmx)
#raise Exception

fig, ax = plt.subplots()
im = ax.imshow(confmx, interpolation='nearest', cmap=cmap)
ax.figure.colorbar(im, ax=ax)
# We want to show all ticks...
ax.set(xticks=np.arange(confmx.shape[1]),
       yticks=np.arange(confmx.shape[0]),
       # ... and label them with the respective list entries
       xticklabels=classes, yticklabels=classes,
       title="Confusion matrix",
       ylabel='True label',
       xlabel='Predicted label')

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
fmt = 'd'
thresh = confmx.max() / 2.
for i in range(confmx.shape[0]):
    for j in range(confmx.shape[1]):
        ax.text(j, i, format(confmx[i, j], fmt),
                ha="center", va="center",
                color="white" if confmx[i, j] > thresh else "black")
fig.tight_layout()